In [36]:
%matplotlib inline
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from collections import Counter
import pandas as pd
import numpy as np
import os
import math
import string
import collections

### Text_Prepare


In [37]:
target_dir = '\\20\\'
base_path = os.getcwd()
print(base_path+target_dir)

C:\Users\iir\Desktop\code\tfidf\20\


In [41]:
import re
from nltk.tokenize import sent_tokenize
def para_prepare(strr):
    """
        text: a string
        return: modified initial string
    """
    originals = nltk.sent_tokenize(strr)
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^a-z\ ]')
    STOPWORDS = set(nltk.corpus.stopwords.words('english'))
    results = []
    for pos, i in enumerate(originals):
        i = i.lower()
        i = REPLACE_BY_SPACE_RE.sub(" ", i)
        i = BAD_SYMBOLS_RE.sub("", i)
        sent = " ".join([w for w in nltk.word_tokenize(i) if w not in STOPWORDS]) 
        results.append(sent)
        
    return results, originals

In [42]:
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self, path):
        self.path = path
        self.files = os.listdir(path)
        self.fn = ""
        
    def getfn(self):
        return self.fn
    
    def __iter__(self):    
        for fn in self.files:
            _data = pd.read_csv(self.path+'/'+fn)
            self.fn = self.path+'/'+fn
            data_size = len(_data)
            for i in range(data_size):
                top = _data['Title'][i]
                para = _data['Abstract'][i]
                """Check abstract is null or not."""
                if type(para) == float and math.isnan(para): 
                    continue
                sent, ori = para_prepare(para)
                for stc, o in zip(sent, ori):
                    if stc == "":
                        continue
                    yield stc, o

In [43]:
sents = MyCorpus(base_path+target_dir)
i = 0
for sent, j in sents:
    i = i + 1
print(i)

1739


### Runingggg

In [44]:
sentences = MyCorpus(base_path+target_dir)

In [45]:
replace_dict = {
    "hepatitis b": "hepatitisb",
}

In [46]:
def words_dict(sents, re_dict, porter = False):
    words = {}
    total_words = 0
    total_sents = 0
    for i, j in sents:
        new_sent = True
        for _old, new_ in re_dict.items():
            i = i.replace(_old, new_)
        token = nltk.word_tokenize(i)
        for pos, t in enumerate(token):
            for _old, new_ in re_dict.items():
                if t == new_:
                    token[pos] = _old
        if porter == False:
            for i in token:
                if i in words:
                    words[i] = words[i] + 1
                else:
                    words[i] = 1
                total_words = total_words + 1
        else:
            stemmer = PorterStemmer()
            for i in token:
                i = stemmer.stem(i)
                if i in words:
                    words[i] = words[i] + 1
                else:
                    words[i] = 1
                total_words = total_words + 1
        total_sents = total_sents + 1
    words = dict(collections.OrderedDict(sorted(words.items())))
    return words, total_words, total_sents
    # return words

In [47]:
tf_nat = lambda x, y: x/y
tf_log = lambda x, y: math.log(1+x)
tf_aug = lambda x, y: 0.5+0.5*((x)/(y))
tf_avg = lambda x, y: ((1+math.log(x))/(1+math.log(y)))
'''-------------------------------------------'''
idf_nat = lambda x, y: math.log(x/y)
idf_smt = lambda x, y: math.log(x/(1+y))+1
idf_max = lambda x, y: math.log(x/(1+y))

In [48]:
words, tw, ts = words_dict(MyCorpus(base_path+target_dir), replace_dict, porter = False)

In [49]:
corpus = MyCorpus(base_path+target_dir)

In [55]:
import pandas as pd
results = {"processed": [], "nonprocessed": []}
for i, j in corpus:
    results["processed"].append(i)
    results["nonprocessed"].append(j)
results_df = pd.DataFrame(results)

results_df.to_csv("./book.csv", header=True, mode='w')

In [52]:
import pandas as pd
results = {"keys": []}
for key in words.keys():
    results["keys"].append(key)
results_df = pd.DataFrame(results)
results_df.to_csv("./keys.csv", header=True, mode='w')

In [53]:
import pandas as pd
''' if you have dict file, please change some code'''
replace_dict = {
    "hepatitis b": "hepatitisb",
}
keys_df = pd.read_csv("./keys.csv")
keys = keys_df["keys"].tolist()
dummy = []
results = {'_keys': keys}
# for key in words.keys():
#     results["keys"].append(key)
# results_df = pd.DataFrame(results)
corpus = MyCorpus(base_path+target_dir)
idx = 0
for i, useless in corpus:
    results[idx] =  [0] * len(keys)
    
    for _old, new_ in replace_dict.items():
        ir = i.replace(_old, new_)
    token = nltk.word_tokenize(ir)
    for pos, t in enumerate(token):
            for _old, new_ in replace_dict.items():
                if t == new_:
                    token[pos] = _old
    for j in token:
        results[idx][keys.index(j)] = results[idx][keys.index(j)] + 1
    idx = idx + 1
results_df = pd.DataFrame(results)
results_df.to_csv("./idf.csv", header=True, mode='w')

In [21]:
# print(results_df)

In [54]:
import pandas as pd
replace_dict = {
    "hepatitis b": "hepatitisb",
}
keys = [i for i, j in words.items()]
dummy = ['_tf', '_ts', '_tw']+keys
results = {'terms': dummy, 'freq': [tw, ts, len(keys)]}
corpus = MyCorpus(base_path+target_dir)
for key, freq in words.items():
    results['freq'].append(freq)
results_df = pd.DataFrame(results)
results_df.to_csv("./tf.csv", header=True, mode='w')